In [13]:
import numpy as np
import tensorflow as tf
import keras
from keras import Sequential
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import os, shutil
from keras.optimizers import * 
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# for file in os.listdir("/content/drive/My Drive/image"):
#   if os.path.isfile(os.path.join("/content/drive/My Drive/image/", file)):
#     shutil.move(os.path.join("/content/drive/My Drive/image/", file),
                # "/content/drive/My Drive/image/train") 

In [ ]:
# for file in os.listdir("/content/drive/My Drive/label"):
#   if os.path.isfile(os.path.join("/content/drive/My Drive/label/", file)):
#     shutil.move(os.path.join("/content/drive/My Drive/label/", file),
#                 "/content/drive/My Drive/label/labels") 

In [ ]:
# shutil.rmtree("/content/gdrive/My Drive/image")
# shutil.rmtree("/content/gdrive/My Drive/label")

# Defining the model architecture &mdash; UNET

The architecture is as follows:

![UNET architecture](https://upload.wikimedia.org/wikipedia/commons/2/2b/Example_architecture_of_U-Net_for_producing_k_256-by-256_image_masks_for_a_256-by-256_RGB_image.png)

According to the [research paper](https://arxiv.org/pdf/1505.04597.pdf)
where this architecture was proposed, after the downsampling section of the network there was a dropout layer included, not explicity shown in the image. Dropout helps with generalization to data, avoiding overfitting.

Data augmentation was also important to favour regularization, discussed later.

In [23]:
def unet(input_size=(256,256,1)):

    inputs = Input(shape=input_size)
    initializer = "glorot_uniform"

    """This is the downsampling feature map part"""

    conv1 = Conv2D(64, 3, padding="same", activation="relu", kernel_initializer=initializer)(inputs)
    conv2 = Conv2D(64, 3, padding="same", activation="relu", kernel_initializer=initializer)(conv1)
    pool1 = MaxPooling2D(2, padding="same")(conv2) #128

    conv3 = Conv2D(128, 3, padding="same", activation="relu", kernel_initializer=initializer)(pool1)
    conv4 = Conv2D(128, 3, padding="same", activation="relu", kernel_initializer=initializer)(conv3)
    pool2 = MaxPooling2D(2, padding="same")(conv4) #64

    conv5 = Conv2D(256, 3, padding="same", activation="relu", kernel_initializer=initializer)(pool2)
    conv6 = Conv2D(256, 3, padding="same", activation="relu", kernel_initializer=initializer)(conv5)
    pool3 = MaxPooling2D(2, padding="same")(conv6) #32

    conv7 = Conv2D(512, 3, padding="same", activation="relu", kernel_initializer=initializer)(pool3)
    conv8 = Conv2D(512, 3, padding="same", activation="relu", kernel_initializer=initializer)(conv7)
    pool4 = MaxPooling2D(2, padding="same")(conv8) #16

    conv9 = Conv2D(1024, 3, padding="same", activation="relu", kernel_initializer=initializer)(pool4)
    conv10 = Conv2D(1024, 3, padding="same", activation="relu", kernel_initializer=initializer)(conv9)

    drop = Dropout(0.5)(conv10)

    """This is the upsampling feature map part"""

    upconv1 = Conv2DTranspose(512, 2, strides=(2,2), padding="same", activation="relu", kernel_initializer=initializer)(drop) #32
    concat1 = Concatenate(axis=-1)([conv8, upconv1])
    conv11 = Conv2D(512, 3, padding="same", activation="relu", kernel_initializer=initializer)(concat1)
    conv12 = Conv2D(512, 3, padding="same", activation="relu", kernel_initializer=initializer)(conv11)

    upconv2 = Conv2DTranspose(256, 2, strides=(2,2), padding="same", activation="relu", kernel_initializer=initializer)(conv12)
    concat2 = Concatenate(axis=-1)([conv6, upconv2])
    conv12 = Conv2D(256, 3, padding="same", activation="relu", kernel_initializer=initializer)(concat2)
    conv13 = Conv2D(256, 3, padding="same", activation="relu", kernel_initializer=initializer)(conv12)

    upconv3 = Conv2DTranspose(128, 2, strides=(2,2), padding="same", activation="relu", kernel_initializer=initializer)(conv13)
    concat3 = Concatenate(axis=-1)([conv4, upconv3])
    conv13 = Conv2D(128, 3, padding="same", activation="relu", kernel_initializer=initializer)(concat3)
    conv14 = Conv2D(128, 3, padding="same", activation="relu", kernel_initializer=initializer)(conv13)

    upconv4 = Conv2DTranspose(64, 2, strides=(2,2), padding="same", activation="relu", kernel_initializer=initializer)(conv14)
    concat4 = Concatenate(axis=-1)([conv2, upconv4])
    conv15 = Conv2D(64, 3, padding="same", activation="relu", kernel_initializer=initializer)(concat4)
    conv16 = Conv2D(64, 3, padding="same", activation="relu", kernel_initializer=initializer)(conv15)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid", kernel_initializer=initializer)(conv16)


    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(0.0001), loss="binary_crossentropy", metrics=["accuracy"])

    
    return model 

In [15]:
def adjustData(img,mask,flag_multi_class,num_class):
    if(flag_multi_class):
        img = img / 255
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
            #for one pixel in the image, find the class in mask and convert it into one-hot vector
            #index = np.where(mask == i)
            #index_mask = (index[0],index[1],index[2],np.zeros(len(index[0]),dtype = np.int64) + i) if (len(mask.shape) == 4) else (index[0],index[1],np.zeros(len(index[0]),dtype = np.int64) + i)
            #new_mask[index_mask] = 1
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):
        img = img / 255
        mask = mask /255
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)

In [16]:
data_gen_args = dict(featurewise_center=False,
                    featurewise_std_normalization=False,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    zoom_range=0.2,
                    horizontal_flip=True)

def trainGenerator(data_aug_dict, seed, train_dir, aug_dir, save_image_prefix,
                   save_mask_prefix, img_dir, mask_dir, flag_multi_class,
                   num_class, class_mode):

  image_datagen = ImageDataGenerator(**data_aug_dict)
  mask_datagen = ImageDataGenerator(**data_aug_dict)
  # Provide the same seed and keyword arguments to the fit and flow methods
  seed = seed
  image_generator = image_datagen.flow_from_directory(
      train_dir,
      classes=[img_dir],
      class_mode=class_mode,
      seed=seed,
      save_to_dir=aug_dir,
      save_prefix=save_image_prefix,
      target_size=(256,256),
      batch_size=20,
      color_mode="grayscale")
  mask_generator = mask_datagen.flow_from_directory(
      train_dir,
      classes=[mask_dir],
      class_mode=class_mode,
      seed=seed,
      save_to_dir=aug_dir,
      save_prefix=save_mask_prefix,
      target_size=(256,256),
      batch_size=20,
      color_mode="grayscale")
  # combine generators into one which yields image and masks
  train_generator = zip(image_generator, mask_generator)

  for (img, mask) in train_generator:
    img, mask = adjustData(img, mask, flag_multi_class, num_class)
    yield (img, mask)

In [17]:
train_gen = trainGenerator(data_gen_args, 147, "/content/drive/My Drive/train/",
                           "/content/drive/My Drive/train/augmented",
                           "image", "mask", "image",
                           "label", False, 2, None)

In [ ]:
  num_batch = 3
for i,batch in enumerate(train_gen):
    if(i >= num_batch):
        break

Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.


In [30]:
chkp = tf.keras.callbacks.ModelCheckpoint(
    filepath="weights.{epoch:02d}-{loss:.2f}.hdf5",
    monitor="loss",
    verbose=1,
    save_best_only=True,
    mode="min",
)

In [24]:
model = unet()
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_82 (Conv2D)              (None, 256, 256, 64) 640         input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_83 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_82[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_17 (MaxPooling2D) (None, 128, 128, 64) 0           conv2d_83[0][0]                  
____________________________________________________________________________________________

Trained the model for 1 epoch: 

```
model.fit_generator(train_gen, steps_per_epoch=2000, epochs=5)

Epoch 1/15
 2000/2000 [===========>] - ETA: 2:11 - loss: 0.1256 - accuracy: 0.9489
```
After that I changed it to:

``` model.fit_generator(train_gen, steps_per_epoch=200, epochs=10)``` 



In [33]:
# model=None
# model = unet()

model.fit_generator(train_gen, steps_per_epoch=200, epochs=10)

Epoch 1/10
200/200 [==============================] - 231s 1s/step - loss: 0.1136 - accuracy: 0.9502
Epoch 2/10
200/200 [==============================] - 236s 1s/step - loss: 0.1097 - accuracy: 0.9520
Epoch 3/10
200/200 [==============================] - 242s 1s/step - loss: 0.1065 - accuracy: 0.9534
Epoch 4/10
200/200 [==============================] - 244s 1s/step - loss: 0.1036 - accuracy: 0.9547
Epoch 5/10
200/200 [==============================] - 249s 1s/step - loss: 0.1004 - accuracy: 0.9561
Epoch 6/10
200/200 [==============================] - 255s 1s/step - loss: 0.0979 - accuracy: 0.9573
Epoch 7/10
200/200 [==============================] - 261s 1s/step - loss: 0.0953 - accuracy: 0.9583
Epoch 8/10
200/200 [==============================] - 267s 1s/step - loss: 0.0933 - accuracy: 0.9593
Epoch 9/10
200/200 [==============================] - 267s 1s/step - loss: 0.0919 - accuracy: 0.9599
Epoch 10/10
200/200 [==============================] - 267s 1s/step - loss: 0.0896 - accura